In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import warnings
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
warnings.filterwarnings('ignore')

# 用convert_dtypes()读取数据

In [2]:
train = pd.read_csv('./input/train.csv').convert_dtypes(convert_string=False)
test = pd.read_csv('./input/testA.csv').convert_dtypes(convert_string=False)

## 定义全局变量

# 先拼train和test 本来想封装一个函数用来拼接
因为似乎concat有个问题，会lose dtype?
复盘了一下M5 感觉M5-simple-fe，def merge-by-concat不太看得懂

### 防呆 这里+col: 'flag'，test的y('isDefault')设为-1

In [3]:
# train.shape  # (800000, 48) & len(train)=800000
# test.shape # (200000, 47)

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [5]:
TARGET = 'isDefault'
train['flag'] = 'train'
test['flag'] = 'test'
test[TARGET] = -1
df = pd.concat([train, test])
# df

In [6]:
# df.info()

### drop policycode, 把flag astype 'string

In [7]:
df.drop('policyCode',axis = 1, inplace=True)

In [8]:
df['flag'].astype('string')

0         train
1         train
2         train
3         train
4         train
          ...  
199995     test
199996     test
199997     test
199998     test
199999     test
Name: flag, Length: 1000000, dtype: string

### issueDate

#### 这里如果string能读datetime吗? 

#### 这里有个全局变量startdate小心

In [9]:
#转化成时间格式，查了确实train的最小日期是2007-6-1，这里加了个issueDateDT,还行吧，就是日期会让数值特别大，

df['issueDate'] = pd.to_datetime(df['issueDate'],format='%Y-%m-%d')
startdate = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
# 构造时间特征
df['issueDateDT'] = df['issueDate'].apply(lambda x: x-startdate).dt.days

In [10]:
pd.set_option('display.max_rows', None)

In [11]:
# df.dtypes   # issueDate datetime64[ns]  '<M8[ns]'

### employmentLength的 years删掉，10+ <1转

In [12]:
def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])

df['employmentLength'].replace(to_replace='10+ years', value='10 years', inplace=True)
df['employmentLength'].replace('< 1 year', '0 years', inplace=True)
df['employmentLength'] = df['employmentLength'].apply(employmentLength_to_int)

In [13]:
df['employmentLength'].value_counts(dropna=False).sort_index()

0.0      80226
1.0      65671
2.0      90565
3.0      80163
4.0      59818
5.0      62645
6.0      46582
7.0      44230
8.0      45168
9.0      37866
10.0    328525
NaN      58541
Name: employmentLength, dtype: int64

#### 这里float，转成Int8

In [14]:
df['employmentTitle'] = df['employmentTitle'].astype('Int8')

In [15]:
# df['employmentTitle'].dtypes  # Int8

### earliesCreditLine 转datetime

#### 这个现在正常了，就是值有点多，不知道要怎么利用抽取信息做特征

之前用excel打开，会变成乱码

In [16]:
df['earliesCreditLine'].sample(10)

196015    Dec-2000
19061     Oct-1995
787886    Sep-1994
270386    Sep-2005
946       Nov-2005
270408    Aug-1989
722243    Nov-2003
679218    Oct-1995
713224    Apr-1985
98882     Aug-2007
95375     Apr-2002
770125    Oct-2003
630686    Jul-2001
41929     May-2004
687233    Dec-2001
275784    Dec-2004
394192    Oct-2002
138796    Nov-1999
640208    Oct-2004
653499    Sep-2004
Name: earliesCreditLine, dtype: object

In [17]:
# dw 这里是提取年信息，我想利用原始信息，顺便还可以describe一下,暂时不用他的
df['earliesCreditLine'] = pd.to_datetime(df['earliesCreditLine'],format='%b-%Y')

In [18]:
pd.set_option('display.max_columns', None)
train.describe(include="all")

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,flag
count,800000.000000,800000.000000,800000.000000,800000.000000,800000.000000,800000,800000,799999.000000,753201,800000.000000,8.000000e+05,800000.000000,800000,800000.000000,800000.000000,799999.000000,800000.000000,799761.000000,800000.000000,800000.000000,800000.000000,800000.000000,800000.000000,799595.000000,8.000000e+05,799469.000000,800000.000000,800000.000000,800000.000000,800000,799999.000000,800000.0,759730.000000,759730.000000,759730.000000,759730.000000,766761.000000,759730.000000,759730.000000,759730.000000,759729.000000,759730.000000,766761.000000,730248.000000,759730.000000,759730.000000,759730.000000,800000
unique,NaN,NaN,NaN,NaN,NaN,7,35,NaN,11,NaN,NaN,NaN,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
top,NaN,NaN,NaN,NaN,NaN,B,C1,NaN,10+ years,NaN,NaN,NaN,2016-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aug-2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
freq,NaN,NaN,NaN,NaN,NaN,233690,50763,NaN,262753,NaN,NaN,NaN,29066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,800000
mean,399999.500000,14416.818875,3.482745,13.238391,437.947723,NaN,NaN,72005.351714,NaN,0.614213,7.613391e+04,1.009683,NaN,0.199513,1.745982,258.535648,16.385758,18.284557,0.318239,696.204081,700.204226,11.598020,0.214915,0.134163,1.622871e+04,51.790734,24.998861,0.416953,0.019267,NaN,1754.113589,1.0,0.511932,3.642330,5.642648,5.642648,4.735641,8.107937,8.575994,8.282953,14.622488,5.592345,11.643896,0.000815,0.003384,0.089366,2.178606,NaN
std,230940.252013,8716.086178,0.855832,4.765757,261.460393,NaN,NaN,106585.640204,NaN,0.675749,6.894751e+04,0.782716,NaN,0.399634,2.367453,200.037446,11.036679,11.150155,0.880325,31.865995,31.866674,5.475286,0.606467,0.377471,2.245802e+04,24.516126,11.999201,0.493055,0.137464,NaN,7941.474040,0.0,1.333266,2.246825,3.302810,3.302810,2.949969,4.799210,7.400536,4.561689,8.124610,3.216184,5.484104,0.030075,0.062041,0.509069,1.844377,NaN
min,0.000000,500.000000,3.000000,5.310000,15.690000,NaN,NaN,0.000000,NaN,0.000000,0.000000e+00,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,630.000000,634.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,2.000000,0.000000,0.000000,NaN,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,199999.750000,8000.000000,3.000000,9.750000,248.450000,NaN,NaN,427.000000,NaN,0.000000,4.560000e+04,0.000000,NaN,0.000000,0.000000,103.000000,8.000000,11.790000,0.000000,670.000000,674.000000,8.000000,0.000000,0.000000,5.944000e+03,33.400000,16.000000,0.000000,0.000000,NaN,0.000000,1.0,0.000000,2.000000,3.000000,3.000000,3.000000,5.000000,4.000000,5.000000,9.000000,3.000000,8.000000,0.000000,0.000000,0.000000,1.000000,NaN
50%,399999.500000,12000.000000,3.000000,12.740000,375.135000,NaN,NaN,7755.000000,NaN,1.000000,6.500000e+04,1.000000,NaN,0.000000,0.000000,203.000000,14.000000,17.610000,0.000000,690.000000,694.000000,11.000000,0.000000,0.000000,1.113200e+04,52.100000,23.000000,0.000000,0.000000,NaN,1.000000,1.0,0.000000,3.000000,5.000000,5.000000,4.000000,7.000000,7.000000,7.000000,13.000000,5.000000,11.000000,0.000000,0.000000,0.000000,2.000000,NaN
75%,599999.250000,20000.000000,3.000000,15.990000,580.710000,NaN,NaN,117663.500000,NaN,1.000000,9.000000e+04,2.000000,NaN,0.000000,4.000000,395.000000,22.000000,24.060000,0

In [19]:
# df.info()

#### 压缩一下内存

In [20]:
import psutil

## Simple "Memory profilers" to see memory usage
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2)

def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [21]:
# 这里改了一些 Int
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['Int16', 'Int32', 'Int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'Int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype('Int8')
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype('Int16')
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype('Int32')
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype('Int64')
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [22]:
print("{:>20}: {:>8}".format('Original df',sizeof_fmt(train.memory_usage(index=True).sum())))

         Original df: 321.2MiB


In [23]:
df = reduce_mem_usage(df)

Mem. usage decreased to 146.87 Mb (63.4% reduction)


In [24]:
# 看下数据类型变没变
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 199999
Data columns (total 48 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   id                  1000000 non-null  Int32         
 1   loanAmnt            1000000 non-null  Int32         
 2   term                1000000 non-null  Int8          
 3   interestRate        1000000 non-null  float16       
 4   installment         1000000 non-null  float16       
 5   grade               1000000 non-null  object        
 6   subGrade            1000000 non-null  object        
 7   employmentTitle     999999 non-null   Int8          
 8   employmentLength    941459 non-null   float16       
 9   homeOwnership       1000000 non-null  Int8          
 10  annualIncome        1000000 non-null  float32       
 11  verificationStatus  1000000 non-null  Int8          
 12  issueDate           1000000 non-null  datetime64[ns]
 13  isDefault    

#### 保存

In [26]:
df.to_csv('cf-df.csv')

In [28]:
import pickle
df.to_pickle('cf-df.pkl')

In [ ]:
# cat_fea = ['grade', 'subGrade', 'employmentTitle', 'verificationStatus', \
#           'purpose', 'postCode', 'regionCode','applicationType','initialListStatus', 'title', 'policyCode']

In [ ]:
# cols = ['id', 'loanAmnt', 'term', 'interestRate', 'installment', 'grade',
#        'subGrade', 'employmentTitle', 'employmentLength', 'homeOwnership',
#        'annualIncome', 'verificationStatus', 'issueDate', 'isDefault',
#        'purpose', 'postCode', 'regionCode', 'dti', 'detlinquency_2years',
#        'ficoRangeLow', 'ficoRangeHigh', 'openAcc', 'pubRec',
#        'pubRecBankruptcies', 'revolBal', 'revolUtil', 'totalAcc',
#        'initialListStatus', 'applicationType', 'earliesCreditLine', 'title',
#        'policyCode', 'n0', 'n1', 'n2', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9',
#        'n10', 'n11', 'n12', 'n13', 'n14']

In [ ]:
# num_fea = [a for a in cols if a not in cat_fea]
# num_fea

In [ ]:
# ## 定义了一个统计函数，方便后续信息统计
# def Sta_inf(data):
#     print('_min',np.min(data))
#     print('_max:',np.max(data))
#     print('_mean',np.mean(data))
#     print('_ptp',np.ptp(data))
#     print('_std',np.std(data))
#     print('_var',np.var(data))